In [1]:
!pip install mlxtend  
%pip install mlxtend --upgrade 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import json
from IPython import display
from urllib.request import urlopen
from io import BytesIO
import matplotlib.pyplot as plt
import seaborn as sns
import itertools 

# Allow multiple output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 


In [3]:
import random

# set seed 
my_seed = 609
random.seed(my_seed)
np.random.seed(my_seed)

In [4]:
# import datasets 
df_delivered = pd.read_csv('dataset_delivered.csv',encoding='utf-8')



In [5]:
data = df_delivered.drop(['level_0','index'],axis=1,)
data = data[data['Телефон_new'] != '0']
data = data[data['Группа2'].notna()]
data.reset_index(inplace=True,drop=True)
data.head()

,Дата,НомерЗаказаНаСайте,НовыйСтатус,Регион,Группа2,Группа3,Группа4,Тип,Номенклатура,ТипТовара,...,Телефон_new,ГородМагазина,МагазинЗаказа,ГодДаты,МесяцДаты,ДеньНеделиДаты,ДеньДаты,ВремяОкруглДаты,СуммаСтроки_v,Маржа_v
0,2017-03-01 12:22:00,3999117_TR,Доставлен,Уральский ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ОДЕЖДА ДЛЯ НОВОРОЖДЕННЫХ (0-2 лет),боди,ОДЕЖДА,"GAMEX, БОДИ кор. рук. Roza, (бел/сер), р. 86, ...",МГТ,...,55575453-56535648535679,0,NaN,2017,3,3,1,12:00:00,379.0,169.2
1,2017-03-01 12:31:00,3999122_TR,Доставлен,Уральский ф.о.,ИГРУШКИ,ИГРУШКИ ДЛЯ ДЕВОЧЕК,имитационные игрушки для девочек,ИГРУШКИ,"ИГРУША, ХОЛОДИЛЬНИК (на бат), (29,5*19*45,5 см)",МГТ,...,55574950-57515657535772,0,NaN,2017,3,3,1,12:00:00,3325.0,2176.0
2,2017-03-01 11:38:00,3999411_TR,Доставлен,Приволжский ф.о.,ИГРУШКИ,ИГРУШКИ ДЛЯ РАЗВИТИЯ МАЛЫШЕЙ,игрушки для ванной,ИГРУШКИ,"S+S TOYS, ИГРУШКА ПВХ в наборе, (2 шт)",МГТ,...,55574854-56495552515179,0,NaN,2017,3,3,1,11:00:00,199.0,120.0
3,2017-03-01 12:19:00,3999682_TR,К отгрузке,Центральный ф.о.,ИГРУШКИ,КОНСТРУКТОРЫ,конструкторы - тип Лего,ИГРУШКИ,"LEGO Duplo, КОНСТРУКТОР Лошадки, (20 дет.)",МГТ,...,55574851-53495753545674,0,NaN,2017,3,3,1,12:00:00,719.0,50.4
4,2017-03-01 12:19:00,3999682_TR,К отгрузке,Центральный ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ДЕТСКАЯ ОДЕЖДА (2-6 лет),леггинсы,ОДЕЖДА,"FUN TIME, ЛОСИНЫ р. 3 г",МГТ,...,55574851-53495753545674,0,NaN,2017,3,3,1,12:00:00,490.0,194.0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373176 entries, 0 to 373175
Data columns (total 30 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Дата                373176 non-null  object 
 1   НомерЗаказаНаСайте  373176 non-null  object 
 2   НовыйСтатус         373176 non-null  object 
 3   Регион              373176 non-null  object 
 4   Группа2             373176 non-null  object 
 5   Группа3             373176 non-null  object 
 6   Группа4             352616 non-null  object 
 7   Тип                 373176 non-null  object 
 8   Номенклатура        373176 non-null  object 
 9   ТипТовара           373176 non-null  object 
 10  Отменено            373176 non-null  object 
 11  Количество          373176 non-null  int64  
 12  Цена                373176 non-null  float64
 13  ЦенаЗакупки         372267 non-null  float64
 14  МесяцДатыЗаказа     373176 non-null  int64  
 15  ГодДатыЗаказа       373176 non-nul

In [7]:
data["Группа2"].value_counts()

ДЕТСКОЕ ПИТАНИЕ              93025
ИГРУШКИ                      73733
ТЕКСТИЛЬ, ТРИКОТАЖ           52116
ПОДГУЗНИКИ                   42521
КОСМЕТИКА/ГИГИЕНА            34599
КРУПНОГАБАРИТНЫЙ ТОВАР       24115
ТОВАРЫ ДЛЯ КОРМЛЕНИЯ         21880
КАНЦТОВАРЫ, КНИГИ, ДИСКИ     15475
ОБУВЬ                         9091
ТОВАРЫ ДЛЯ ЖИВОТНЫХ           5267
СОПУТСТВУЮЩИЕ ТОВАРЫ           729
ЖЕНСКИЕ ШТУЧКИ                 600
ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА       25
Name: Группа2, dtype: int64

In [8]:
cols = ['Телефон_new']
cols.extend(data['Группа2'].unique())
MB = pd.DataFrame(columns=cols)


MB 


,Телефон_new,"ТЕКСТИЛЬ, ТРИКОТАЖ",ИГРУШКИ,КРУПНОГАБАРИТНЫЙ ТОВАР,ОБУВЬ,СОПУТСТВУЮЩИЕ ТОВАРЫ,ПОДГУЗНИКИ,КОСМЕТИКА/ГИГИЕНА,ЖЕНСКИЕ ШТУЧКИ,"КАНЦТОВАРЫ, КНИГИ, ДИСКИ",ТОВАРЫ ДЛЯ КОРМЛЕНИЯ,ДЕТСКОЕ ПИТАНИЕ,ТОВАРЫ ДЛЯ ЖИВОТНЫХ,ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА


In [9]:
# pd.pivot_table(data[data['МесяцДаты']==5][data['ДеньДаты']==1][['Телефон_new','Группа2']], index='Телефон_new', columns='Группа2', aggfunc=len, fill_value=0).reset_index()

In [10]:
# Calculate transactions
def get_transactions(data):
  cols = ['Телефон_new']
  cols.extend(data['Группа2'].unique())
  MB = pd.DataFrame(columns=cols)

  range_of_months = data['МесяцДаты'].unique()
  for i in range_of_months:
    range_of_days = data[data['МесяцДаты']==i]['ДеньДаты'].unique()

    for j in range_of_days:
      current_day_pv = pd.pivot_table(data[data['МесяцДаты']==i][data['ДеньДаты']==j][['Телефон_new','Группа2']], index='Телефон_new', columns='Группа2', aggfunc=len, fill_value=0).reset_index()
      MB = pd.concat([MB,current_day_pv]).fillna(0)

  MB.reset_index(drop=True, inplace=True)
  return MB


# Transactions - relative to unique client
def customer_transactions(data):
    return pd.pivot_table(data[['Телефон_new','Группа2']], 
                        index='Телефон_new', columns='Группа2', 
                        aggfunc=len, fill_value=0).reset_index(drop=True)

def transform_mb(MB):
  cols = ['ДЕТСКОЕ ПИТАНИЕ', 'ЖЕНСКИЕ ШТУЧКИ', 'ИГРУШКИ',
       'КАНЦТОВАРЫ, КНИГИ, ДИСКИ', 'КОСМЕТИКА/ГИГИЕНА',
       'КРУПНОГАБАРИТНЫЙ ТОВАР', 'ОБУВЬ', 'ПОДГУЗНИКИ', 'СОПУТСТВУЮЩИЕ ТОВАРЫ',
       'ТЕКСТИЛЬ, ТРИКОТАЖ', 'ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА',
       'ТОВАРЫ ДЛЯ ЖИВОТНЫХ', 'ТОВАРЫ ДЛЯ КОРМЛЕНИЯ']
  MB['number_of_checks'] = MB[cols].astype(bool).sum(axis=1)
  return MB[MB['number_of_checks'] == 2].drop(['number_of_checks'],axis=1).astype(int).reset_index(drop=True)




In [11]:
MB = get_transactions(data)
MB

,Телефон_new,"ТЕКСТИЛЬ, ТРИКОТАЖ",ИГРУШКИ,КРУПНОГАБАРИТНЫЙ ТОВАР,ОБУВЬ,СОПУТСТВУЮЩИЕ ТОВАРЫ,ПОДГУЗНИКИ,КОСМЕТИКА/ГИГИЕНА,ЖЕНСКИЕ ШТУЧКИ,"КАНЦТОВАРЫ, КНИГИ, ДИСКИ",ТОВАРЫ ДЛЯ КОРМЛЕНИЯ,ДЕТСКОЕ ПИТАНИЕ,ТОВАРЫ ДЛЯ ЖИВОТНЫХ,ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА
0,55525753-53565350484974,0,0,0,0,0.0,1,0,0.0,0,0,0,0,0.0
1,55574848-48495057545270,3,0,0,0,0.0,0,0,0.0,0,0,0,0,0.0
2,55574848-51545054504876,0,0,0,1,0.0,0,0,0.0,0,0,0,0,0.0
3,55574848-52575748515375,1,0,0,0,0.0,0,0,0.0,0,0,0,0,0.0
4,55574848-53555456495070,0,0,1,0,0.0,0,0,0.0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110651,55575757-48555648505178,0,0,0,0,0.0,0,0,0.0,0,0,5,0,0.0
110652,55575757-49515753495279,1,0,0,0,0.0,0,0,0.0,0,0,0,0,0.0
110653,55575757-54574957515179,0,0,1,0,0.0,0,0,0.0,0,0,0,0,0.0
110654,55575757-56484857554877,0,1,0,0,0.0,0,0,0.0,0,0,0,0,0.0


In [12]:
MB.to_csv('MBClient.csv', index=False, encoding='utf-8') 

In [13]:
MB

,Телефон_new,"ТЕКСТИЛЬ, ТРИКОТАЖ",ИГРУШКИ,КРУПНОГАБАРИТНЫЙ ТОВАР,ОБУВЬ,СОПУТСТВУЮЩИЕ ТОВАРЫ,ПОДГУЗНИКИ,КОСМЕТИКА/ГИГИЕНА,ЖЕНСКИЕ ШТУЧКИ,"КАНЦТОВАРЫ, КНИГИ, ДИСКИ",ТОВАРЫ ДЛЯ КОРМЛЕНИЯ,ДЕТСКОЕ ПИТАНИЕ,ТОВАРЫ ДЛЯ ЖИВОТНЫХ,ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА
0,55525753-53565350484974,0,0,0,0,0.0,1,0,0.0,0,0,0,0,0.0
1,55574848-48495057545270,3,0,0,0,0.0,0,0,0.0,0,0,0,0,0.0
2,55574848-51545054504876,0,0,0,1,0.0,0,0,0.0,0,0,0,0,0.0
3,55574848-52575748515375,1,0,0,0,0.0,0,0,0.0,0,0,0,0,0.0
4,55574848-53555456495070,0,0,1,0,0.0,0,0,0.0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110651,55575757-48555648505178,0,0,0,0,0.0,0,0,0.0,0,0,5,0,0.0
110652,55575757-49515753495279,1,0,0,0,0.0,0,0,0.0,0,0,0,0,0.0
110653,55575757-54574957515179,0,0,1,0,0.0,0,0,0.0,0,0,0,0,0.0
110654,55575757-56484857554877,0,1,0,0,0.0,0,0,0.0,0,0,0,0,0.0


In [15]:
MB_ = pd.read_csv('MBClient.csv') 
MB_

,Телефон_new,"ТЕКСТИЛЬ, ТРИКОТАЖ",ИГРУШКИ,КРУПНОГАБАРИТНЫЙ ТОВАР,ОБУВЬ,СОПУТСТВУЮЩИЕ ТОВАРЫ,ПОДГУЗНИКИ,КОСМЕТИКА/ГИГИЕНА,ЖЕНСКИЕ ШТУЧКИ,"КАНЦТОВАРЫ, КНИГИ, ДИСКИ",ТОВАРЫ ДЛЯ КОРМЛЕНИЯ,ДЕТСКОЕ ПИТАНИЕ,ТОВАРЫ ДЛЯ ЖИВОТНЫХ,ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА
0,55525753-53565350484974,0,0,0,0,0.0,1,0,0.0,0,0,0,0,0.0
1,55574848-48495057545270,3,0,0,0,0.0,0,0,0.0,0,0,0,0,0.0
2,55574848-51545054504876,0,0,0,1,0.0,0,0,0.0,0,0,0,0,0.0
3,55574848-52575748515375,1,0,0,0,0.0,0,0,0.0,0,0,0,0,0.0
4,55574848-53555456495070,0,0,1,0,0.0,0,0,0.0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110651,55575757-48555648505178,0,0,0,0,0.0,0,0,0.0,0,0,5,0,0.0
110652,55575757-49515753495279,1,0,0,0,0.0,0,0,0.0,0,0,0,0,0.0
110653,55575757-54574957515179,0,0,1,0,0.0,0,0,0.0,0,0,0,0,0.0
110654,55575757-56484857554877,0,1,0,0,0.0,0,0,0.0,0,0,0,0,0.0


In [16]:
MB = MB_

In [17]:
MB = MB.drop(['Телефон_new'], axis=1)


MB = MB.applymap(lambda x: 1 if x >= 1 else 0)

# columns_to_sum =  ['КРУПНОГАБАРИТНЫЙ ТОВАР', 'ТЕКСТИЛЬ, ТРИКОТАЖ', 'ОБУВЬ',
#        'ИГРУШКИ', 'КАНЦТОВАРЫ, КНИГИ, ДИСКИ', 'ТОВАРЫ ДЛЯ КОРМЛЕНИЯ',
#        'СОПУТСТВУЮЩИЕ ТОВАРЫ', 'КОСМЕТИКА/ГИГИЕНА', 'ЖЕНСКИЕ ШТУЧКИ',
#        'ПОДГУЗНИКИ', 'ДЕТСКОЕ ПИТАНИЕ', 'ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА',
#        'ТОВАРЫ ДЛЯ ЖИВОТНЫХ']


columns_to_sum = [col for col in MB.columns if col != 'Телефон_new']


MB['sum_cols'] = MB[columns_to_sum].sum(axis=1)

MB = MB[MB['sum_cols'] == 2]

MB = MB.drop(['sum_cols'], axis=1)

MB.reset_index(drop=True,inplace=True)

In [18]:
MB

,"ТЕКСТИЛЬ, ТРИКОТАЖ",ИГРУШКИ,КРУПНОГАБАРИТНЫЙ ТОВАР,ОБУВЬ,СОПУТСТВУЮЩИЕ ТОВАРЫ,ПОДГУЗНИКИ,КОСМЕТИКА/ГИГИЕНА,ЖЕНСКИЕ ШТУЧКИ,"КАНЦТОВАРЫ, КНИГИ, ДИСКИ",ТОВАРЫ ДЛЯ КОРМЛЕНИЯ,ДЕТСКОЕ ПИТАНИЕ,ТОВАРЫ ДЛЯ ЖИВОТНЫХ,ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА
0,0,1,0,0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,1,0,0
4,1,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19715,0,0,0,0,0,1,1,0,0,0,0,0,0
19716,0,0,0,0,0,1,0,0,0,1,0,0,0
19717,0,0,0,0,0,1,1,0,0,0,0,0,0
19718,0,1,0,0,0,0,0,0,0,1,0,0,0


In [19]:
params = ["support", "confidence", "lift", "expected confidence"] 
res = pd.DataFrame(columns=params) 

In [20]:
cols.remove('Телефон_new')
permutations = {tuple(sorted(p)) for p in itertools.permutations(cols, 2)}
rows = [' + '.join(i) for i in permutations] 
pairs = list(permutations)

In [21]:
import random
print(f"There are {len(pairs)} pairs") 
print(*random.choices(rows,k=5),sep='\n') 

There are 78 pairs
ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА + ТОВАРЫ ДЛЯ ЖИВОТНЫХ
ИГРУШКИ + ТОВАРЫ ДЛЯ ЖИВОТНЫХ
ДЕТСКОЕ ПИТАНИЕ + КОСМЕТИКА/ГИГИЕНА
КОСМЕТИКА/ГИГИЕНА + ПОДГУЗНИКИ
ЖЕНСКИЕ ШТУЧКИ + ИГРУШКИ


In [22]:
temp = pd.DataFrame(0, index=[i for i in rows], columns=params) 

res = res.append(temp, ignore_index=False, sort=False) 
res = res.reset_index() 
res = res.rename(columns={"index": "Pairs"}) 


In [23]:
def support(a, b, df):
    counter = 0
    for i in range(0, len(df)):
        # print(a)
        # print(df)
        if df[a][i] > 0 and df[b][i] > 0:
            counter += 1
    return counter/len(df) 

def confidence(a, b, df):
    counter = 0
    total = 0
    for i in range(0, len(df)):
        if df[a][i] > 0:
            total += 1 
            if df[b][i] > 0:
                counter += 1
    return counter/total 

def expconf(a, b, df):
    counter = 0
    for i in range(0, len(df)):
        if df[b][i] > 0:
            counter += 1
    return counter/len(df) 

In [24]:
for i in range(len(pairs)): 
    sup = support(pairs[i][0], pairs[i][1], MB) 
    conf = confidence(pairs[i][0], pairs[i][1], MB) 
    exconf = expconf(pairs[i][0], pairs[i][1], MB) 
    lift = conf / exconf 
    res['support'][i] = sup 
    res['confidence'][i] = conf 
    res['expected confidence'][i] = exconf 
    res['lift'][i] = lift 

In [25]:
res.sort_values('support',ascending=False)

,Pairs,support,confidence,lift,expected confidence
16,ДЕТСКОЕ ПИТАНИЕ + ПОДГУЗНИКИ,0.149645,0.555221,1.312353,0.423073
50,КОСМЕТИКА/ГИГИЕНА + ПОДГУЗНИКИ,0.095994,0.476346,1.12592,0.423073
61,"ИГРУШКИ + ТЕКСТИЛЬ, ТРИКОТАЖ",0.073073,0.197967,0.844818,0.234331
39,"ИГРУШКИ + КАНЦТОВАРЫ, КНИГИ, ДИСКИ",0.072008,0.195082,1.886715,0.103398
46,ИГРУШКИ + ПОДГУЗНИКИ,0.064249,0.174062,0.411424,0.423073
...,...,...,...,...,...
23,ДЕТСКОЕ ПИТАНИЕ + ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА,0.0,0.0,0.0,0.000051
22,ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА + ТОВАРЫ ДЛЯ КОРМЛЕНИЯ,0.0,0.0,0.0,0.145385
14,ДЕТСКОЕ ПИТАНИЕ + СОПУТСТВУЮЩИЕ ТОВАРЫ,0.0,0.0,0.0,0.011207
9,ОБУВЬ + ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА,0.0,0.0,0.0,0.000051


In [26]:
res.to_csv('MBClientFinal.csv', index=False, encoding='utf-8') 

# FP-Growth 

In [27]:
from mlxtend.preprocessing import TransactionEncoder 
from mlxtend.frequent_patterns import association_rules, fpgrowth, apriori, fpmax, hmine
warnings.filterwarnings("ignore", category=DeprecationWarning) 


In [28]:
dfp = MB.applymap(lambda x: True if x >= 1 else False)

In [29]:
frequent_itemsets_fp = fpgrowth(dfp, min_support=0.001, use_colnames=True)	


In [30]:
frequent_itemsets_fp.sort_values('support', ascending=False)


,support,itemsets
0,0.423073,(ПОДГУЗНИКИ)
1,0.369118,(ИГРУШКИ)
4,0.269523,(ДЕТСКОЕ ПИТАНИЕ)
2,0.234331,"(ТЕКСТИЛЬ, ТРИКОТАЖ)"
7,0.201521,(КОСМЕТИКА/ГИГИЕНА)
24,0.149645,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
6,0.145385,(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)
5,0.135041,(КРУПНОГАБАРИТНЫЙ ТОВАР)
8,0.103398,"(КАНЦТОВАРЫ, КНИГИ, ДИСКИ)"
37,0.095994,"(КОСМЕТИКА/ГИГИЕНА, ПОДГУЗНИКИ)"


# Association rules

In [31]:
rules_ = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=0.001)

In [34]:
rules_.sort_values('support', ascending=False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
24,(ПОДГУЗНИКИ),(ДЕТСКОЕ ПИТАНИЕ),0.423073,0.269523,0.149645,0.353710,1.312353,0.035617,1.130261,0.412548
25,(ДЕТСКОЕ ПИТАНИЕ),(ПОДГУЗНИКИ),0.269523,0.423073,0.149645,0.555221,1.312353,0.035617,1.297110,0.325828
51,(ПОДГУЗНИКИ),(КОСМЕТИКА/ГИГИЕНА),0.423073,0.201521,0.095994,0.226897,1.125920,0.010736,1.032823,0.193850
50,(КОСМЕТИКА/ГИГИЕНА),(ПОДГУЗНИКИ),0.201521,0.423073,0.095994,0.476346,1.125920,0.010736,1.101734,0.140063
4,"(ТЕКСТИЛЬ, ТРИКОТАЖ)",(ИГРУШКИ),0.234331,0.369118,0.073073,0.311837,0.844818,-0.013423,0.916763,-0.193486
5,(ИГРУШКИ),"(ТЕКСТИЛЬ, ТРИКОТАЖ)",0.369118,0.234331,0.073073,0.197967,0.844818,-0.013423,0.954660,-0.225502
59,"(КАНЦТОВАРЫ, КНИГИ, ДИСКИ)",(ИГРУШКИ),0.103398,0.369118,0.072008,0.696420,1.886715,0.033842,2.078141,0.524177
58,(ИГРУШКИ),"(КАНЦТОВАРЫ, КНИГИ, ДИСКИ)",0.369118,0.103398,0.072008,0.195082,1.886715,0.033842,1.113905,0.744954
0,(ИГРУШКИ),(ПОДГУЗНИКИ),0.369118,0.423073,0.064249,0.174062,0.411424,-0.091914,0.698512,-0.693964
1,(ПОДГУЗНИКИ),(ИГРУШКИ),0.423073,0.369118,0.064249,0.151864,0.411424,-0.091914,0.743846,-0.712616


# Apriori

In [35]:
apriori_res = apriori(dfp, min_support=0.001, use_colnames=True).sort_values("support",ascending=False)

In [36]:
apriori_res.sort_values('support', ascending=False)


,support,itemsets
5,0.423073,(ПОДГУЗНИКИ)
1,0.369118,(ИГРУШКИ)
10,0.269523,(ДЕТСКОЕ ПИТАНИЕ)
0,0.234331,"(ТЕКСТИЛЬ, ТРИКОТАЖ)"
6,0.201521,(КОСМЕТИКА/ГИГИЕНА)
45,0.149645,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
9,0.145385,(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)
2,0.135041,(КРУПНОГАБАРИТНЫЙ ТОВАР)
8,0.103398,"(КАНЦТОВАРЫ, КНИГИ, ДИСКИ)"
42,0.095994,"(КОСМЕТИКА/ГИГИЕНА, ПОДГУЗНИКИ)"


# FPMax

In [37]:
fpmax_res = fpmax(dfp, min_support=0.001, use_colnames=True)

In [38]:
fpmax_res.sort_values('support', ascending=False)


,support,itemsets
42,0.149645,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
37,0.095994,"(КОСМЕТИКА/ГИГИЕНА, ПОДГУЗНИКИ)"
40,0.073073,"(ТЕКСТИЛЬ, ТРИКОТАЖ, ИГРУШКИ)"
22,0.072008,"(ИГРУШКИ, КАНЦТОВАРЫ, КНИГИ, ДИСКИ)"
43,0.064249,"(ИГРУШКИ, ПОДГУЗНИКИ)"
28,0.050000,"(ИГРУШКИ, КРУПНОГАБАРИТНЫЙ ТОВАР)"
15,0.041278,"(ТЕКСТИЛЬ, ТРИКОТАЖ, ОБУВЬ)"
33,0.037170,"(ПОДГУЗНИКИ, ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)"
39,0.035294,"(ТЕКСТИЛЬ, ТРИКОТАЖ, ПОДГУЗНИКИ)"
36,0.035193,"(КОСМЕТИКА/ГИГИЕНА, ДЕТСКОЕ ПИТАНИЕ)"


# Hmine

In [39]:
hmine_res = hmine(dfp, min_support=0.001,use_colnames=True,verbose=False)


In [40]:
hmine_res.sort_values('support', ascending=False)


,support,itemsets
35,0.423073,(ПОДГУЗНИКИ)
11,0.369118,(ИГРУШКИ)
53,0.269523,(ДЕТСКОЕ ПИТАНИЕ)
0,0.234331,"(ТЕКСТИЛЬ, ТРИКОТАЖ)"
41,0.201521,(КОСМЕТИКА/ГИГИЕНА)
39,0.149645,"(ДЕТСКОЕ ПИТАНИЕ, ПОДГУЗНИКИ)"
51,0.145385,(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)
21,0.135041,(КРУПНОГАБАРИТНЫЙ ТОВАР)
48,0.103398,"(КАНЦТОВАРЫ, КНИГИ, ДИСКИ)"
36,0.095994,"(КОСМЕТИКА/ГИГИЕНА, ПОДГУЗНИКИ)"


# MBA based on Clusters

In [40]:
def process_clusters(df, num_of_clusters):
  results = []
  
  
  
  for cluster in range(num_of_clusters):
    local_resuts = {
                "FP-Growth":None,
                "Association rules":None,
                "Apriori":None,
                "FPMax":None,
                "Hmine":None}
    data = df

    data = data[data['Labels'] == cluster]

    cols = ['Телефон_new']
    cols.extend(data['Группа2'].unique())
    MB = pd.DataFrame(columns=cols)

    #
    MB = get_transactions(data)
    MB_ = MB

    MB = MB.drop(['Телефон_new'], axis=1)


    MB = MB.applymap(lambda x: 1 if x >= 1 else 0)

    # columns_to_sum =  ['КРУПНОГАБАРИТНЫЙ ТОВАР', 'ТЕКСТИЛЬ, ТРИКОТАЖ', 'ОБУВЬ',
    #        'ИГРУШКИ', 'КАНЦТОВАРЫ, КНИГИ, ДИСКИ', 'ТОВАРЫ ДЛЯ КОРМЛЕНИЯ',
    #        'СОПУТСТВУЮЩИЕ ТОВАРЫ', 'КОСМЕТИКА/ГИГИЕНА', 'ЖЕНСКИЕ ШТУЧКИ',
    #        'ПОДГУЗНИКИ', 'ДЕТСКОЕ ПИТАНИЕ', 'ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА',
    #        'ТОВАРЫ ДЛЯ ЖИВОТНЫХ']


    columns_to_sum = [col for col in MB.columns if col != 'Телефон_new']


    MB['sum_cols'] = MB[columns_to_sum].sum(axis=1)

    MB = MB[MB['sum_cols'] == 2]

    MB = MB.drop(['sum_cols'], axis=1)

    MB.reset_index(drop=True,inplace=True)

    #


  

    # FP-Growth
    dfp = MB.drop(['Телефон_new'], axis=1, errors='ignore').applymap(lambda x: True if x >= 1 else False)
    frequent_itemsets_fp = fpgrowth(dfp, min_support=0.001, use_colnames=True)	
    
    local_resuts['FP-Growth'] = frequent_itemsets_fp
    
    # association rules
    rules_fp = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=0.02)

    local_resuts['Association rules'] = rules_fp

    #Apriori
    apriori_res = apriori(dfp, min_support=0.001, use_colnames=True).sort_values("support",ascending=False)

    local_resuts['Apriori'] = apriori_res

    # fpmax
    fpmax_res = fpmax(dfp, min_support=0.001, use_colnames=True)

    local_resuts['FPMax'] = fpmax_res

    #hmine
    hmine_res = hmine(dfp, min_support=0.001,use_colnames=True,verbose=False)

    local_resuts['Hmine'] = hmine_res

    results.append(local_resuts)

  return results

In [41]:
df_c = pd.read_csv('df_12_clusters_km.csv')

In [42]:
df_c['Labels'].dropna(inplace=True)

In [43]:
df_c.head()

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,target,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка
0,55574954-50515053535277,992,0.44,318,3,2,1.0,0.0,0.0,0.0,...,1,435,1,145.00,435,3,327.36,0.0,327.36,327.36
1,55574851-50484949535070,1698,0.26,849,2,2,1.0,0.0,0.0,0.0,...,0,444,1,222.00,444,2,0.00,0.0,0.00,1698.00
2,55574954-56545048505170,4038,0.28,281,18,9,1.0,0.0,0.0,0.0,...,0,1132,1,62.89,1132,19,242.28,0.0,2907.36,0.00
3,55574954-53555555545178,1280,0.39,427,2,3,1.0,0.0,0.0,0.0,...,0,501,1,250.50,501,3,0.00,0.0,0.00,1280.00
4,55574857-54544951504973,6381,0.14,456,14,14,1.0,0.0,0.0,0.0,...,0,921,1,65.79,921,17,446.67,0.0,1340.01,893.34


In [44]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79389 entries, 0 to 79388
Data columns (total 46 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Телефон_new                            79389 non-null  object 
 1   Выручка_bought                         79389 non-null  int64  
 2   Маржа_bought                           79389 non-null  float64
 3   Средний чек_bought                     79389 non-null  int64  
 4   Кол-во чеков_bought                    79389 non-null  int64  
 5   Среднее кол-во товаров в  чеке_bought  79389 non-null  int64  
 6   Центральный ф.о.                       79389 non-null  float64
 7   Приволжский ф.о.                       79389 non-null  float64
 8   Северо-Западный ф.о.                   79389 non-null  float64
 9   Южный ф.о.                             79389 non-null  float64
 10  Сибирский ф.о.                         79389 non-null  float64
 11  Ур

In [45]:
phone_cluster = df_c[['Телефон_new','Labels']].drop_duplicates()


In [46]:
phone_cluster

,Телефон_new,Labels
0,55574954-50515053535277,9
1,55574851-50484949535070,4
2,55574954-56545048505170,9
3,55574954-53555555545178,4
4,55574857-54544951504973,3
...,...,...
79384,55574949-57565751545277,9
79385,55574854-55505648555577,1
79386,55575053-50544856505076,4
79387,55575054-55574849485475,9


In [132]:
phone_cluster.reset_index(inplace=True,drop=True)


In [48]:
data_c = df_delivered.drop(['level_0','index'],axis=1,)
data_c = data_c[data_c['Телефон_new'] != '0']
data_c = data_c[data_c['Группа2'].notna()]
data_c.reset_index(inplace=True,drop=True)

data_c = data_c.merge(df_c[['Телефон_new','Labels']], on='Телефон_new')




In [49]:
data_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322790 entries, 0 to 322789
Data columns (total 31 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Дата                322790 non-null  object 
 1   НомерЗаказаНаСайте  322790 non-null  object 
 2   НовыйСтатус         322790 non-null  object 
 3   Регион              322790 non-null  object 
 4   Группа2             322790 non-null  object 
 5   Группа3             322790 non-null  object 
 6   Группа4             306291 non-null  object 
 7   Тип                 322790 non-null  object 
 8   Номенклатура        322790 non-null  object 
 9   ТипТовара           322790 non-null  object 
 10  Отменено            322790 non-null  object 
 11  Количество          322790 non-null  int64  
 12  Цена                322790 non-null  float64
 13  ЦенаЗакупки         321882 non-null  float64
 14  МесяцДатыЗаказа     322790 non-null  int64  
 15  ГодДатыЗаказа       322790 non-nul

In [50]:
all_clusters = process_clusters(data_c, len(df_c.Labels.unique()))

In [51]:
all_clusters[3]['Apriori']

,support,itemsets
5,0.652904,(ДЕТСКОЕ ПИТАНИЕ)
4,0.592722,(ПОДГУЗНИКИ)
36,0.381386,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
3,0.289713,(КОСМЕТИКА/ГИГИЕНА)
6,0.151155,(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)
31,0.129811,"(ПОДГУЗНИКИ, КОСМЕТИКА/ГИГИЕНА)"
2,0.127362,(ИГРУШКИ)
7,0.088174,"(ТЕКСТИЛЬ, ТРИКОТАЖ)"
32,0.084325,"(ДЕТСКОЕ ПИТАНИЕ, КОСМЕТИКА/ГИГИЕНА)"
40,0.073478,"(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ, ДЕТСКОЕ ПИТАНИЕ)"


In [52]:
import pickle


def cluster_mba_save(all_clusters):
    with open("all_clusters.pickle", 'wb') as f:
        pickle.dump(all_clusters, f, pickle.HIGHEST_PROTOCOL)

def cluster_mba_load(path):
    with open('path', 'rb') as f:
        return pickle.load(f)

cluster_mba_save(all_clusters)

# Recommendation systems

In [41]:
!pip install scikit-surprise

In [42]:
# SVD

In [43]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold


In [44]:
from collections import defaultdict
# Official function. 
# Reference: https://surprise.readthedocs.io/en/stable/FAQ.html?highlight=top#how-to-get-the-top-n-recommendations-for-each-user

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [45]:
data

,Дата,НомерЗаказаНаСайте,НовыйСтатус,Регион,Группа2,Группа3,Группа4,Тип,Номенклатура,ТипТовара,...,Телефон_new,ГородМагазина,МагазинЗаказа,ГодДаты,МесяцДаты,ДеньНеделиДаты,ДеньДаты,ВремяОкруглДаты,СуммаСтроки_v,Маржа_v
0,2017-03-01 12:22:00,3999117_TR,Доставлен,Уральский ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ОДЕЖДА ДЛЯ НОВОРОЖДЕННЫХ (0-2 лет),боди,ОДЕЖДА,"GAMEX, БОДИ кор. рук. Roza, (бел/сер), р. 86, ...",МГТ,...,55575453-56535648535679,0,NaN,2017,3,3,1,12:00:00,379.0,169.20
1,2017-03-01 12:31:00,3999122_TR,Доставлен,Уральский ф.о.,ИГРУШКИ,ИГРУШКИ ДЛЯ ДЕВОЧЕК,имитационные игрушки для девочек,ИГРУШКИ,"ИГРУША, ХОЛОДИЛЬНИК (на бат), (29,5*19*45,5 см)",МГТ,...,55574950-57515657535772,0,NaN,2017,3,3,1,12:00:00,3325.0,2176.00
2,2017-03-01 11:38:00,3999411_TR,Доставлен,Приволжский ф.о.,ИГРУШКИ,ИГРУШКИ ДЛЯ РАЗВИТИЯ МАЛЫШЕЙ,игрушки для ванной,ИГРУШКИ,"S+S TOYS, ИГРУШКА ПВХ в наборе, (2 шт)",МГТ,...,55574854-56495552515179,0,NaN,2017,3,3,1,11:00:00,199.0,120.00
3,2017-03-01 12:19:00,3999682_TR,К отгрузке,Центральный ф.о.,ИГРУШКИ,КОНСТРУКТОРЫ,конструкторы - тип Лего,ИГРУШКИ,"LEGO Duplo, КОНСТРУКТОР Лошадки, (20 дет.)",МГТ,...,55574851-53495753545674,0,NaN,2017,3,3,1,12:00:00,719.0,50.40
4,2017-03-01 12:19:00,3999682_TR,К отгрузке,Центральный ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ДЕТСКАЯ ОДЕЖДА (2-6 лет),леггинсы,ОДЕЖДА,"FUN TIME, ЛОСИНЫ р. 3 г",МГТ,...,55574851-53495753545674,0,NaN,2017,3,3,1,12:00:00,490.0,194.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373171,2017-04-17 03:18:00,4410609_TR,Доставлен,Южный ф.о.,ПОДГУЗНИКИ,ПОДГУЗНИКИ,подгузники,ППКП,"DORAEMON, ПОДГУЗНИКИ (9-14 кг)",МГТ,...,55575449-53485056485678,0,NaN,2017,4,1,17,03:00:00,865.0,185.00
373172,2017-04-17 10:10:00,4411268_TR,Доставлен,Центральный ф.о.,ИГРУШКИ,ИГРУШКИ ДЛЯ РАЗВИТИЯ МАЛЫШЕЙ,прочие игрушки для развития,ИГРУШКИ,"TOMY LAMAZE, КНИЖКА- шуршалка Пират Пит",МГТ,...,55575054-53545152544971,0,NaN,2017,4,1,17,10:00:00,464.0,139.20
373173,2017-04-17 11:03:00,4411571_TR,Доставлен,Приволжский ф.о.,КОСМЕТИКА/ГИГИЕНА,ПРЕДМЕТЫ ПО УХОДУ ЗА НОВОРОЖДЕННЫМИ,щетки зубные,ППКП,"CS MEDICA, НАСАДКА для зубной щетки SonicPulsa...",МГТ,...,55575448-49564957485078,0,NaN,2017,4,1,17,11:00:00,249.0,42.00
373174,2017-04-17 11:03:00,4411571_TR,Доставлен,Приволжский ф.о.,КОСМЕТИКА/ГИГИЕНА,ПРЕДМЕТЫ ПО УХОДУ ЗА НОВОРОЖДЕННЫМИ,щетки зубные,ППКП,"CS MEDICA, ЩЕТКА зубная эл. SonicPulsar CS-562...",МГТ,...,55575448-49564957485078,0,NaN,2017,4,1,17,11:00:00,849.0,144.00


In [46]:
ones = pd.DataFrame(data.shape[0]*[1],columns=['present'])
data_rs = data[['Телефон_new','Группа4']]
data_rs = pd.concat([data_rs, ones],axis=1)
data_rs

,Телефон_new,Группа4,present
0,55575453-56535648535679,боди,1
1,55574950-57515657535772,имитационные игрушки для девочек,1
2,55574854-56495552515179,игрушки для ванной,1
3,55574851-53495753545674,конструкторы - тип Лего,1
4,55574851-53495753545674,леггинсы,1
...,...,...,...
373171,55575449-53485056485678,подгузники,1
373172,55575054-53545152544971,прочие игрушки для развития,1
373173,55575448-49564957485078,щетки зубные,1
373174,55575448-49564957485078,щетки зубные,1


In [47]:
# # Create a reader specifying the rating scale
reader = Reader(rating_scale=(0, 1))  # Assuming binary presence of goods in the bill

# # Load the dataset into Surprise
dataset = Dataset.load_from_df(data_rs, reader)

In [50]:
kf = KFold(n_splits=5)
algo = SVD()

for trainset, testset in kf.split(dataset):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.0
0.0


0.0
0.0


0.0
0.0


0.0
0.0


0.0
0.0


In [49]:
# # NMF
# from surprise import NMF


In [69]:
!pip install tabulate

In [51]:
from surprise.model_selection import cross_validate, KFold
from tabulate import tabulate
import datetime
import time
from surprise import (
    BaselineOnly,
    CoClustering,
    Dataset,
    KNNBaseline,
    KNNBasic,
    KNNWithMeans,
    NMF,
    NormalPredictor,
    SlopeOne,
    SVD,
    SVDpp,
)

In [52]:
kf = KFold(random_state=609)  # folds will be the same for all algorithms.


In [53]:

# The algorithms to cross-validate
algos = (
    SVD(random_state=609),
    SVDpp(random_state=609, cache_ratings=False),
    SVDpp(random_state=609, cache_ratings=True),
    NMF(random_state=609),

    # KNNBasic(),
#    KNNWithMeans(),
#    KNNBaseline(),
#    CoClustering(random_state=609),
#    SlopeOne(),
#    BaselineOnly(),
#    NormalPredictor(),
 
)

algo_results = pd.DataFrame(columns=["Name","RMSE","MAE","TIME"])
algo_results = algo_results.set_index("Name")



for algo in algos:
    start = time.time()
    out = cross_validate(algo, dataset, ["rmse", "mae"], kf)
    cv_time = str(datetime.timedelta(seconds=int(time.time() - start)))
    link = algo.__class__.__name__
    cache_ratings=False
    try: 
        cache_ratings = algo.__getattribute__("cache_ratings")
    except:
        cache_ratings=False

    if cache_ratings:
        link += " w/ cache_ratings"
    mean_rmse = "{:.3f}".format(np.mean(out["test_rmse"]))
    mean_mae = "{:.3f}".format(np.mean(out["test_mae"]))
    new_line = {"Name" :link, "RMSE":mean_rmse, "MAE": mean_mae,  "TIME": cv_time}
    print(tabulate([new_line], tablefmt="pipe"))  # print current algo perf
    algo_results = algo_results.append(new_line, ignore_index=True)


algo_results.sort_values('RMSE',ascending=True)

|:----|------:|------:|:--------|
| SVD | 0.012 | 0.003 | 0:00:12 |
|:------|------:|------:|:--------|
| SVDpp | 0.004 | 0.001 | 0:00:24 |
|:-----------------------|------:|------:|:--------|
| SVDpp w/ cache_ratings | 0.004 | 0.001 | 0:15:55 |
|:----|------:|------:|:--------|
| NMF | 0.061 | 0.057 | 0:00:24 |


,RMSE,MAE,TIME,Name
1,0.004,0.001,0:00:24,SVDpp
2,0.004,0.001,0:15:55,SVDpp w/ cache_ratings
0,0.012,0.003,0:00:12,SVD
3,0.061,0.057,0:00:24,NMF


In [179]:
algo_results.sort_values('RMSE',ascending=True)

,RMSE,MAE,TIME,Name
1,0.004,0.001,0:00:24,SVDpp
2,0.004,0.001,0:00:23,SVDpp w/ cache_ratings
0,0.011,0.003,0:00:13,SVD
3,0.062,0.057,0:00:24,NMF


In [103]:

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
trainset = dataset.build_full_trainset()

algo = SVD()
algo.fit(trainset)

testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# # Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    break

55574954-50515053535277 ['комбинезоны', 'носки', 'песочники', 'боди', 'комплекты, наборы', 'матрасы в кровать', 'пляжная обувь', 'качели', 'кровати', 'куклы/пупсы']


# Now run on each cluster

In [104]:
data_c

,Дата,НомерЗаказаНаСайте,НовыйСтатус,Регион,Группа2,Группа3,Группа4,Тип,Номенклатура,ТипТовара,...,ГородМагазина,МагазинЗаказа,ГодДаты,МесяцДаты,ДеньНеделиДаты,ДеньДаты,ВремяОкруглДаты,СуммаСтроки_v,Маржа_v,Labels
0,2017-05-01 00:03:00,4513479_TR,Доставлен,Центральный ф.о.,КРУПНОГАБАРИТНЫЙ ТОВАР,ИЗДЕЛИЯ ИЗ ПЛАСТМАССЫ,горшки,КГТ,"ТЕГА, ГОРШОК Веселая Черепаха, Польша",КГТ,...,0,NaN,2017,5,1,1,00:00:00,152.0,76.00,9
1,2017-05-01 00:03:00,4513479_TR,Доставлен,Центральный ф.о.,ОБУВЬ,ОБУВЬ ДЕТСКАЯ,резиновая обувь,ОДЕЖДА,"ДЮНА, РЕЗИНА неутепленная, (сер), р. 23-26",МГТ,...,0,NaN,2017,5,1,1,00:00:00,560.0,245.00,9
2,2017-06-18 22:15:00,4930289_TR,Доставлен,Центральный ф.о.,ИГРУШКИ,ИГРУШКИ ДЛЯ РАЗВИТИЯ МАЛЫШЕЙ,прочие игрушки для развития,ИГРУШКИ,"АЗБУКВАРИК, СМАРТФОН Песенки В.Шаинского, (в асс)",МГТ,...,0,NaN,2017,6,7,18,22:00:00,280.0,113.68,9
3,2017-05-01 10:18:00,4513492_TR,К отгрузке,Центральный ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ДЕТСКАЯ ОДЕЖДА (7-16 лет),жилеты,ОДЕЖДА,"LEADER KIDS, ЖИЛЕТ Тигр, (син), р.140",МГТ,...,0,NaN,2017,5,1,1,10:00:00,499.0,199.00,4
4,2017-05-01 10:18:00,4513492_TR,К отгрузке,Центральный ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ДЕТСКАЯ ОДЕЖДА (7-16 лет),джемпера,ОДЕЖДА,"S`COOL, ТОЛСТОВКА, (сер.мел+черн+бирюза), р.140",МГТ,...,0,NaN,2017,5,1,1,10:00:00,1199.0,245.00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322785,2017-06-30 20:21:00,5030177_TR,Доставлен,Центральный ф.о.,ОБУВЬ,ОБУВЬ ДЕТСКАЯ,текстильная обувь,ОДЕЖДА,"КОТОФЕЙ, ТУФЛИ (син), р. 20-26",МГТ,...,0,NaN,2017,6,5,30,20:00:00,740.0,305.00,4
322786,2017-06-30 20:21:00,5030177_TR,Доставлен,Центральный ф.о.,ОБУВЬ,ОБУВЬ ДЕТСКАЯ,текстильная обувь,ОДЕЖДА,"КОТОФЕЙ, ТУФЛИ (сер), р. 19-22",МГТ,...,0,NaN,2017,6,5,30,20:00:00,870.0,435.00,4
322787,2017-06-30 20:21:00,5030177_TR,Доставлен,Центральный ф.о.,ОБУВЬ,ОБУВЬ ДЕТСКАЯ,текстильная обувь,ОДЕЖДА,"КОТОФЕЙ, ТУФЛИ (син+бел), р. 19-22",МГТ,...,0,NaN,2017,6,5,30,20:00:00,870.0,435.00,4
322788,2017-06-30 20:21:00,5030297_TR,Доставлен,Центральный ф.о.,ИГРУШКИ,ИГРУШКИ ДЛЯ РАЗВИТИЯ МАЛЫШЕЙ,игрушки из дерева,ИГРУШКИ,"МИР ДЕРЕВЯННЫХ ИГРУШЕК, ДЕРГУНЧИК Нюша",МГТ,...,0,NaN,2017,6,5,30,20:00:00,225.0,85.50,9


In [183]:
def recommender_systems_by_cluster(df, number_of_cluster, verbose = True):
    results = []
    for cluster in range(number_of_cluster):
        print('|'*6, '\tCluster', cluster, '|'*6)
        # Prepare Data
        # only selected cluster
        data = df[df['Labels']==cluster].reset_index(drop=True)
        ones = pd.DataFrame(data.shape[0]*[1],columns=['present'])
        data_rs = data[['Телефон_new','Группа4']]
        data_rs = pd.concat([data_rs, ones],axis=1)
        # print(data_rs)


        # # Create a reader specifying the rating scale
        reader = Reader(rating_scale=(0, 1))  # Assuming binary presence of goods in the bill

        # # Load the dataset into Surprise
        dataset = Dataset.load_from_df(data_rs, reader)
        
        # save metrics
        algo_results = pd.DataFrame(columns=["Name","RMSE","MAE","TIME"])
        algo_results = algo_results.set_index("Name")

        # The algorithms to cross-validate
        algos = (
            SVD(random_state=609),
            SVDpp(random_state=609, cache_ratings=False),
            SVDpp(random_state=609, cache_ratings=True),
            NMF(random_state=609),

            # KNNBasic(),
        #    KNNWithMeans(),
        #    KNNBaseline(),
        #    CoClustering(random_state=609),
        #    SlopeOne(),
        #    BaselineOnly(),
        #    NormalPredictor(),
        
        )


        for algo in algos:
            start = time.time()
            out = cross_validate(algo, dataset, ["rmse", "mae"], kf)
            cv_time = str(datetime.timedelta(seconds=int(time.time() - start)))
            link = algo.__class__.__name__
            cache_ratings=False
            try: 
                cache_ratings = algo.__getattribute__("cache_ratings")
            except:
                cache_ratings=False

            if cache_ratings:
                link += " w/ cache_ratings"
            mean_rmse = "{:.3f}".format(np.mean(out["test_rmse"]))
            mean_mae = "{:.3f}".format(np.mean(out["test_mae"]))
            new_line = {"Name" :link, "RMSE":mean_rmse, "MAE": mean_mae,  "TIME": cv_time}
            print(tabulate([new_line], tablefmt="pipe"))  # print current algo perf
            algo_results = algo_results.append(new_line, ignore_index=True)

        results.append(algo_results)
        print('\n')
    
    return results
        

In [184]:
output = recommender_systems_by_cluster(data_c, len(df_c.Labels.unique()))

|||||| 	Cluster 0 ||||||
|:----|------:|-----:|:--------|
| SVD | 0.042 | 0.02 | 0:00:00 |
|:------|-----:|-----:|:--------|
| SVDpp | 0.02 | 0.01 | 0:00:00 |
|:-----------------------|-----:|-----:|:--------|
| SVDpp w/ cache_ratings | 0.02 | 0.01 | 0:00:00 |
|:----|------:|------:|:--------|
| NMF | 0.066 | 0.057 | 0:00:23 |


|||||| 	Cluster 1 ||||||
|:----|------:|------:|:--------|
| SVD | 0.024 | 0.008 | 0:00:02 |
|:------|------:|------:|:--------|
| SVDpp | 0.011 | 0.003 | 0:00:02 |
|:-----------------------|------:|------:|:--------|
| SVDpp w/ cache_ratings | 0.011 | 0.003 | 0:00:02 |
|:----|------:|------:|:--------|
| NMF | 0.064 | 0.059 | 0:00:04 |


|||||| 	Cluster 2 ||||||
|:----|-----:|------:|:--------|
| SVD | 0.02 | 0.006 | 0:00:00 |
|:------|------:|------:|:--------|
| SVDpp | 0.014 | 0.005 | 0:00:00 |
|:-----------------------|------:|------:|:--------|
| SVDpp w/ cache_ratings | 0.014 | 0.005 | 0:00:00 |
|:----|------:|------:|:--------|
| NMF | 0.055 | 0.026 | 0

# Tables for the report

In [208]:
output[0][['RMSE','Name']].set_index('Name')

,RMSE
Name,
SVD,0.042
SVDpp,0.020
SVDpp w/ cache_ratings,0.020
NMF,0.066


In [214]:
temp = pd.DataFrame()

for i in output:
    temp = pd.concat([temp,i[['RMSE','Name']].set_index('Name')],axis=1)

In [216]:
temp.T

Name,SVD,SVDpp,SVDpp w/ cache_ratings,NMF
RMSE,0.042,0.020,0.020,0.066
RMSE,0.024,0.011,0.011,0.064
RMSE,0.020,0.014,0.014,0.055
RMSE,0.023,0.009,0.009,0.061
RMSE,0.026,0.011,0.011,0.065
RMSE,0.028,0.014,0.014,0.063
RMSE,0.044,0.031,0.031,0.073
RMSE,0.039,0.020,0.020,0.066
RMSE,0.039,0.019,0.019,0.091
RMSE,0.024,0.010,0.010,0.064


In [174]:
group0 = 3

all_clusters[group0]['Apriori'].sort_values('support',ascending=False).reset_index(drop=True)[0:10]

,support,itemsets
0,0.652904,(ДЕТСКОЕ ПИТАНИЕ)
1,0.592722,(ПОДГУЗНИКИ)
2,0.381386,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
3,0.289713,(КОСМЕТИКА/ГИГИЕНА)
4,0.151155,(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)
5,0.129811,"(ПОДГУЗНИКИ, КОСМЕТИКА/ГИГИЕНА)"
6,0.127362,(ИГРУШКИ)
7,0.088174,"(ТЕКСТИЛЬ, ТРИКОТАЖ)"
8,0.084325,"(ДЕТСКОЕ ПИТАНИЕ, КОСМЕТИКА/ГИГИЕНА)"
9,0.073478,"(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ, ДЕТСКОЕ ПИТАНИЕ)"


In [175]:
all_clusters[group0]['FP-Growth'].sort_values('support',ascending=False).reset_index(drop=True)[0:10]

,support,itemsets
0,0.652904,(ДЕТСКОЕ ПИТАНИЕ)
1,0.592722,(ПОДГУЗНИКИ)
2,0.381386,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
3,0.289713,(КОСМЕТИКА/ГИГИЕНА)
4,0.151155,(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)
5,0.129811,"(ПОДГУЗНИКИ, КОСМЕТИКА/ГИГИЕНА)"
6,0.127362,(ИГРУШКИ)
7,0.088174,"(ТЕКСТИЛЬ, ТРИКОТАЖ)"
8,0.084325,"(ДЕТСКОЕ ПИТАНИЕ, КОСМЕТИКА/ГИГИЕНА)"
9,0.073478,"(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ, ДЕТСКОЕ ПИТАНИЕ)"


In [170]:
all_clusters[group0]['Association rules'].reset_index(drop=True)[0:10]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(ПОДГУЗНИКИ),(ДЕТСКОЕ ПИТАНИЕ),0.592722,0.652904,0.381386,0.643447,0.985516,-0.005605,0.973477,-0.034829
1,(ДЕТСКОЕ ПИТАНИЕ),(ПОДГУЗНИКИ),0.652904,0.592722,0.381386,0.584137,0.985516,-0.005605,0.979356,-0.040622
2,(ПОДГУЗНИКИ),"(ТЕКСТИЛЬ, ТРИКОТАЖ)",0.592722,0.088174,0.013296,0.022432,0.254409,-0.038966,0.932750,-0.877986
3,"(ТЕКСТИЛЬ, ТРИКОТАЖ)",(ПОДГУЗНИКИ),0.088174,0.592722,0.013296,0.150794,0.254409,-0.038966,0.479598,-0.762700
4,"(ТЕКСТИЛЬ, ТРИКОТАЖ)",(КОСМЕТИКА/ГИГИЕНА),0.088174,0.289713,0.012946,0.146825,0.506796,-0.012599,0.832522,-0.516274
5,(КОСМЕТИКА/ГИГИЕНА),"(ТЕКСТИЛЬ, ТРИКОТАЖ)",0.289713,0.088174,0.012946,0.044686,0.506796,-0.012599,0.954478,-0.578081
6,(ИГРУШКИ),"(ТЕКСТИЛЬ, ТРИКОТАЖ)",0.127362,0.088174,0.011896,0.093407,1.059349,0.000666,1.005772,0.064201
7,"(ТЕКСТИЛЬ, ТРИКОТАЖ)",(ИГРУШКИ),0.088174,0.127362,0.011896,0.134921,1.059349,0.000666,1.008738,0.061442
8,"(ТЕКСТИЛЬ, ТРИКОТАЖ)",(ДЕТСКОЕ ПИТАНИЕ),0.088174,0.652904,0.034640,0.392857,0.601707,-0.022929,0.571687,-0.420608
9,(ДЕТСКОЕ ПИТАНИЕ),"(ТЕКСТИЛЬ, ТРИКОТАЖ)",0.652904,0.088174,0.034640,0.053055,0.601707,-0.022929,0.962914,-0.656012


In [171]:
all_clusters[group0]['FPMax'].sort_values('support',ascending=False).reset_index(drop=True)[0:10]

,support,itemsets
0,0.381386,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
1,0.129811,"(ПОДГУЗНИКИ, КОСМЕТИКА/ГИГИЕНА)"
2,0.084325,"(ДЕТСКОЕ ПИТАНИЕ, КОСМЕТИКА/ГИГИЕНА)"
3,0.073478,"(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ, ДЕТСКОЕ ПИТАНИЕ)"
4,0.045836,"(ИГРУШКИ, ДЕТСКОЕ ПИТАНИЕ)"
5,0.034640,"(ТЕКСТИЛЬ, ТРИКОТАЖ, ДЕТСКОЕ ПИТАНИЕ)"
6,0.033940,"(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ, КОСМЕТИКА/ГИГИЕНА)"
7,0.029391,"(ПОДГУЗНИКИ, ИГРУШКИ)"
8,0.022743,"(ПОДГУЗНИКИ, ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)"
9,0.021693,"(ДЕТСКОЕ ПИТАНИЕ, КРУПНОГАБАРИТНЫЙ ТОВАР)"


In [172]:
all_clusters[group0]['Hmine'].sort_values('support',ascending=False).reset_index(drop=True)[0:10]

,support,itemsets
0,0.652904,(ДЕТСКОЕ ПИТАНИЕ)
1,0.592722,(ПОДГУЗНИКИ)
2,0.381386,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
3,0.289713,(КОСМЕТИКА/ГИГИЕНА)
4,0.151155,(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)
5,0.129811,"(ПОДГУЗНИКИ, КОСМЕТИКА/ГИГИЕНА)"
6,0.127362,(ИГРУШКИ)
7,0.088174,"(ТЕКСТИЛЬ, ТРИКОТАЖ)"
8,0.084325,"(ДЕТСКОЕ ПИТАНИЕ, КОСМЕТИКА/ГИГИЕНА)"
9,0.073478,"(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ, ДЕТСКОЕ ПИТАНИЕ)"
